In [40]:
# https://www.geeksforgeeks.org/sparse-matrix-representations-set-3-csr/
struct MyCSR
    m::Int64                  # Number of rows
    n::Int64                  # Number of columns
    rowptr::Vector{Int64}     # Row j is in rowptr[j]:(rowptr[j+1]-1)
    colval::Vector{Int64}     # Col indices of stored values
    nzval::Vector{Float64}    # Stored values, typically nonzeros
    function MyCSR(m::Int64, n::Int64)
        rowptr=zeros(Int64, m+1)
        colval=Vector{Int64}()
        nzval=Vector{Float64}()
        return new(m, n,rowptr,colval,nzval)
    end
    MyCSR(m,n,rowptr,colval,nzval)=new(m,n,rowptr,colval, nzval)
end
# CSR is good! remember me!!!!!!
# Base.getindex(::MyCSR, i::Int64, j::Int64) = 3



In [66]:
function Densify(a::MyCSR)
    M=zeros(Float64, a.m, a.n)
    for row in 1:a.m
        for i in a.rowptr[row]:a.rowptr[row+1]-1
            c=a.colval[i]
            M[row, c]=a.nzval[i]
        end
    end
    return M
end

Densify (generic function with 1 method)

In [86]:
function adjoint(m::MyCSR)
    return m
end

adjoint (generic function with 1 method)

In [33]:
A=MyCSR(3,5)

MyCSR(3, 5, [0, 0, 0, 0], Int64[], Float64[])

In [34]:
using LinearAlgebra;
using Latexify;

In [35]:
dummy = Dict{String, Array{String}}("a"=>["b","c"], "b"=>["a"], "c"=>["a","d"], "d"=>["b"])

Dict{String, Array{String, N} where N} with 4 entries:
  "c" => ["a", "d"]
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [36]:
input=dummy

Dict{String, Array{String, N} where N} with 4 entries:
  "c" => ["a", "d"]
  "b" => ["a"]
  "a" => ["b", "c"]
  "d" => ["b"]

In [72]:
damping=0.85
function naive_get_matrix(exitLinks)
    n=length(dummy)
    encodeNames=String[]
    decodeNames=Dict{String, Int64}()
    i=1
    for (key, value) in exitLinks
        push!(encodeNames, key)
        push!(decodeNames, key=>i)
        i+=1
    end
    
    
    
    
    rowptr=ones(Int64, n+1)
    colval=Vector{Int64}()
    nzval=Vector{Float64}()
    i=2
    for (from, list) in exitLinks 
        f=decodeNames[from]
        l=length(list)
        rowptr[i]=rowptr[i-1]+l
        for j in 1:l 
            t=decodeNames[list[j]]
            append!(nzval, 1/l)
            append!(colval, t)
        end
        i+=1
    end
    return (MyCSR(n,n, rowptr, colval, nzval), decodeNames)
end

naive_get_matrix (generic function with 1 method)

In [73]:
(S, dec)=naive_get_matrix(input)

(MyCSR(4, 4, [1, 3, 4, 6, 7], [3, 4, 3, 2, 1, 2], [0.5, 0.5, 1.0, 0.5, 0.5, 1.0]), Dict("c" => 1, "b" => 2, "a" => 3, "d" => 4))

In [75]:
dec

Dict{String, Int64} with 4 entries:
  "c" => 1
  "b" => 2
  "a" => 3
  "d" => 4

In [80]:
M=Densify(S)

4×4 Matrix{Float64}:
 0.0  0.0  0.5  0.5
 0.0  0.0  1.0  0.0
 0.5  0.5  0.0  0.0
 0.0  1.0  0.0  0.0

In [90]:
adjoint(S)

MyCSR(4, 4, [1, 3, 4, 6, 7], [3, 4, 3, 2, 1, 2], [0.5, 0.5, 1.0, 0.5, 0.5, 1.0])